In [1]:

import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import numpy as np


In [2]:
# Load data
stations = gpd.read_file('/Users/vinaymadhugiri/Downloads/NJTransit_Rail_Stations_-3939820597305918471.geojson')  # NJ Transit stations GeoJSON
lines = gpd.read_file('/Users/vinaymadhugiri/Downloads/Rail_Lines_of_NJ_Transit.geojson')   # Rail lines GeoJSON
municipalities = gpd.read_file('/Users/vinaymadhugiri/Downloads/NJ_Municipal_Boundaries_3424_930771950455602469.geojson')  # Municipalities GeoJSON

# Create a Folium map centered around a specific location (e.g., New Jersey)
nj_map = folium.Map(location=[40.0583, -74.4057], zoom_start=9)

In [3]:
# Perform spatial joins
stations_with_lines = gpd.sjoin(stations, lines, how="left", predicate="intersects")

# Rename the index columns if they exist
stations_with_lines = stations_with_lines.rename(columns={'index_left': 'left_index', 'index_right': 'right_index'})
municipalities = municipalities.rename(columns={'index_left': 'left_index', 'index_right': 'right_index'})

# Now perform the second spatial join
stations_with_data = gpd.sjoin(stations_with_lines, municipalities, how="left", predicate="within")

In [4]:
print(stations_with_data.head(25))

    OBJECTID_1            STATION_ID COUNTY_left  LATITUDE  LONGITUDE  \
0            1  Point Pleasant Beach       Ocean  40.09272  -74.04819   
1            2           Spring Lake    Monmouth  40.15057  -74.03546   
2            3                Belmar    Monmouth  40.18059  -74.02730   
3            4         Bradley Beach    Monmouth  40.20378  -74.01896   
4            5           Asbury Park    Monmouth  40.21536  -74.01479   
5            6            Allenhurst    Monmouth  40.23766  -74.00676   
6            7               Elberon    Monmouth  40.26532  -73.99759   
7            8           Long Branch    Monmouth  40.29715  -73.98834   
8            9         Monmouth Park    Monmouth  40.31339  -74.01519   
9           10         Little Silver    Monmouth  40.32671  -74.04105   
10          11              Red Bank    Monmouth  40.34831  -74.07455   
11          12            Middletown    Monmouth  40.38978  -74.11613   
12          13                Hazlet    Monmouth  4

In [5]:
# Add rail lines
folium.GeoJson(lines, name="Rail Lines").add_to(nj_map)

In [6]:
# Add Font Awesome CSS to the map
nj_map.get_root().header.add_child(folium.Element('<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">'))

# Add stations to the map
for idx, row in stations_with_data.iterrows():
    popup_text = f"""
    <b>Station Name:</b> {row['STATION_ID']}<br>
    <b>Rail Line:</b> {row['LINE_CODE_left']}<br>
    <b>Municipality:</b> {row['MUN_LABEL']}<br>
    <b>Population:</b> {row['POP2020']:,}
    """
    icon_html = '<i class="fa-solid fa-train" style="font-size: 20px; color: black;"></i>'
    folium.Marker(
    location=[row.geometry.y, row.geometry.x],
    popup=folium.Popup(popup_text, max_width=300),
    icon=folium.DivIcon(html=icon_html)
).add_to(nj_map)

In [7]:

# Save the map to an HTML file
nj_map.save('Visual_1.html')
